## 환경 세팅

In [ ]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install bitsandbytes
!pip install accelerate
!pip install accelerate --upgrade
!pip install --upgrade typing_extensions

In [2]:
!pip install -U pip

!pip install \
  torch \
  transformers \
  accelerate \
  peft \
  langchain \
  faiss-cpu \
  gradio \
  sentence-transformers \
  huggingface_hub \
  langchain-community \
  langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 295.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 119.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 210.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 257.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 427.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 169.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 427.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 193.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# ✅ numpy 1.26 이상에서만 sentence-transformers가 정상 동작하므로 버전 명시
!pip install --upgrade numpy==1.26.4 scipy scikit-learn --quiet

# ✅ 벡터 검색 및 임베딩용!
!pip install sentence-transformers --quiet

# ✅ LangChain 핵심 + community 기능
!pip install langchain langchain-community --quiet

# ✅ huggingface 모델 로딩 + gradio UI + FAISS용
!pip install transformers accelerate gradio faiss-cpu --quiet

---

## 파인튜닝

In [ ]:
import os
import json
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer, BitsAndBytesConfig
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from huggingface_hub import login

# ✅ Hugging Face 로그인 (토큰 입력)
login("")

# ✅ CUDA 환경 설정 (GPU 1개만 사용)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# ✅ 데이터 로드
with open("./data/cleaned_familylaw_finetune_data.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)
dataset = Dataset.from_list(raw_data)

# ✅ 모델/토크나이저 로드
model_id = "openchat/openchat-3.5-0106"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# ✅ 전처리 함수 정의
def format_prompt(example):
    prompt = f"### 질문:\n{example['instruction']}\n\n### 문서:\n{example['input']}\n\n### 답변:"
    response = example["output"]
    full_text = prompt + " " + response

    # 전체 시퀀스를 하나로 처리
    tokenized = tokenizer(
        full_text,
        truncation=True,
        padding="max_length",
        max_length=1024,
    )

    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    # 레이블은 input_ids 복사해서 답변 전까지는 마스킹
    labels = input_ids.copy()

    # prompt 길이만큼 -100 마스킹
    prompt_len = len(tokenizer(prompt, truncation=True, max_length=1024)["input_ids"])
    labels[:prompt_len] = [-100] * prompt_len

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

tokenized_dataset = dataset.map(format_prompt)

# ✅ QLoRA 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# ✅ 모델 로드 (GPU 0만 사용)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
    low_cpu_mem_usage=True
)
base_model = prepare_model_for_kbit_training(base_model)

# ✅ LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(base_model, lora_config)

# ✅ 학습 인자
training_args = TrainingArguments(
    output_dir="./qlora_openchat_familylaw",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    save_strategy="epoch",
    fp16=True,
    logging_steps=10,
    report_to="none"
)

# ✅ Trainer 정의 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

# ✅ 모델 저장
model.save_pretrained("./qlora_openchat_familylaw_v2/peft_model")
tokenizer.save_pretrained("./qlora_openchat_familylaw_v2/peft_model")

# ✅ 파인튜닝 모델 테스트용 추론 함수
def infer(instruction, context=""):
    model.eval()
    prompt = f"### 질문:\n{instruction}\n\n### 문서:\n{context}\n\n### 답변:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.3,
            top_p=0.85,
            eos_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✅ 예시 추론
print(infer("친권과 양육권의 차이점은 무엇인가요?"))

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/tmp/ipykernel_2851/742583032.py:102: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for mod

Step,Training Loss
10,0.724800
20,0.622600
30,0.485500
40,0.487200
50,0.491400
60,0.348300


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


### 질문:
친권과 양육권의 차이점은 무엇인가요?

### 문서:


### 답변: 친권과 양육권은 가족 관계에서 두 가지 다른 권한을 나타내는 개념입니다.

친권은 가족 간의 감정적 관계와 의존성을 나타내는 것으로, 가족 구성원들 간의 정신적 지지와 사랑을 의미합니다. 이는 가족 구성원들이 서로 친밀하게 지내며 서로에 대한 감정적 연결을 유지하는 것을 의미합니다.

양육권은 가족 구성원들이 자녀에게 책임을 지고 양육 역할을 수행하는 데 필요한 권한을 의미합니다. 이는 가족 구성원들


## 모델 저장

In [9]:
from huggingface_hub import upload_folder

repo_id = "skyss/skn-3rd"

# ✅ 저장된 모델 경로
folder_path = "./qlora_openchat_familylaw_v2/peft_model"

# ✅ 업로드 실행
upload_folder(
    repo_id=repo_id,
    folder_path=folder_path,
    path_in_repo=".",  # 루트에 업로드
    commit_message="🚀 Upload QLoRA PEFT fine-tuned family law model"
)

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/skyss/skn-3rd/commit/c70caecd2325764ac7b53ca6b91c55af457ea91a', commit_message='🚀 Upload QLoRA PEFT fine-tuned family law model', commit_description='', oid='c70caecd2325764ac7b53ca6b91c55af457ea91a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/skyss/skn-3rd', endpoint='https://huggingface.co', repo_type='model', repo_id='skyss/skn-3rd'), pr_revision=None, pr_num=None)

### 사용자 화면 및 프롬프트 설정

In [ ]:
import os
import torch
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from datetime import datetime
import uuid
from openai import OpenAI
import re
import numpy as np

# ✅ 환경 설정

# ✅ 모델 및 벡터 DB
model_path = "./qlora_openchat_familylaw_v2/peft_model"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).to(device)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device.type == "cuda" else -1)

embedding = OpenAIEmbeddings()
vectordb = FAISS.load_local("law_case_db", embeddings=embedding, allow_dangerous_deserialization=True)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# ✅ 사용자 정보 초기값
user_info = {
    "user_id": str(uuid.uuid4()),
    "marital_status": "",
    "marriage_duration": "",
    "divorce_stage": "",
    "children": "",
    "abuse_history": "",
    "property_range": "",
    "private_question": False,
    "created_at": datetime.utcnow(),
    "registered": False
}
chat_history = []

# ✅ 텍스트 후처리 함수

def clean_text(text):
    return text.replace("\n", " ").replace("  ", " ").strip()


# ✅ UTF-8 안전 처리 함수
def safe_utf8(text):
    return text.encode("utf-8", "surrogatepass").decode("utf-8", "ignore")

def register_user(marital_status, m_priv, marriage_duration, d_priv, divorce_stage, ds_priv,
                  children, c_priv, abuse_history, a_priv, property_range, p_priv, private_question):
    user_info.update({
        "marital_status": "비공개" if m_priv else marital_status,
        "marriage_duration": "비공개" if d_priv else marriage_duration,
        "divorce_stage": "비공개" if ds_priv else divorce_stage,
        "children": "비공개" if c_priv else children,
        "abuse_history": "비공개" if a_priv else abuse_history,
        "property_range": "비공개" if p_priv else property_range,
        "private_question": private_question,
        "registered": True
    })
    return "✅ 사용자 정보가 성공적으로 등록되었습니다!"

def deduplicate_text(text):
    lines = text.splitlines()
    seen = set()
    cleaned = []
    for line in lines:
        line = line.strip()
        if line and line not in seen:
            cleaned.append(line)
            seen.add(line)
    return "\n".join(cleaned)


def format_documents_with_metadata(docs):
    formatted_docs = []
    for i, doc in enumerate(docs):
        meta = doc.metadata
        title = meta.get("사건번호") or meta.get("조항번호") or meta.get("법률명") or meta.get("판례일련번호") or f"문서 {i+1}"
        content = doc.page_content.strip()
        formatted_docs.append(f"📄 문서 {i+1} ({title}):\n{content}")
    return "\n\n".join(formatted_docs)

def remove_trailing_phrases(text):
    phrases = ["최대 1000자 이내로 요약되었습니다.", "요약을 마칩니다.", "이상으로 요약합니다."]
    for p in phrases:
        if p in text:
            text = text.replace(p, "")
    return text.strip()

def format_answer_blocks(text):
    # 1. 법적 해석과 답변 생성이 없다면 요약 생성
    if "### 1. 법적 해석과 답변" not in text:
        gpt_response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "다음 내용은 법적 해석에 대한 요약입니다. 법률적 근거와 판례만 남기고 실무 전략 및 개인적 고려는 제외하세요."},
                {"role": "user", "content": text + "\n\n법적 근거와 판례를 중심으로 요약해주세요."}
            ]
        )
        legal_analysis = gpt_response.choices[0].message.content.strip()
        text = f"### 1. ⚖️ 법적 해석과 답변\n{legal_analysis}"

    # 2. 맞춤형 대응 전략 생성이 없다면 생성
    if "### 2. 맞춤형 대응 전략" not in text:
        gpt_response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "다음 내용은 맞춤형 대응 전략입니다. 이미 제공된 법적 해석(1번)과 중복되지 않도록 실무적 접근과 구체적 대응 방안을 중심으로 작성하세요."},
                {"role": "user", "content": text + "\n\n법적 해석을 제외하고 맞춤형 대응 전략을 작성해주세요."}
            ]
        )
        gpt_strategy = gpt_response.choices[0].message.content.strip()
        text += f"\n\n### 2. 🛠 맞춤형 대응 전략\n{gpt_strategy}"

    # 마크 구분 삽입
    if "🟠" not in text and "🟢" not in text and "🟡" not in text:
        if "OpenAI GPT" in text:
            text += "\n\n🟡 (OpenAI GPT)"
        else:
            text += "\n\n🟢 (파인튜닝 모델)"

    lines = text.split("\n")
    processed = []
    for i, line in enumerate(lines):
        if line.strip() == "":
            processed.append("")
        else:
            if i + 1 < len(lines) and lines[i + 1].strip() != "":
                processed.append(line + "\n")
            else:
                processed.append(line)
    text = "".join(processed)

    return text.strip()


def contains_excessive_repetition(text):
    patterns = [
        r"(따라서 .*? 사건의 특성에 따라.*?[\.\n]){2,}",
        r"(이 답변은 .*? 전문가와 상담하는 것이 좋습니다[\.\n]){2,}",
        r"(법적 조언이 필요합니다[\.\n]){2,}",
        r"(이에 따라 .*? 결정됩니다[\.\n]){2,}",
        r"(이러한 경우 .*? 법률 전문가와 상의하시기 바랍니다[\.\n]){2,}",
        r"(결론적으로 .*? 달라질 수 있습니다[\.\n]){2,}",
        r"(이러한 사건에 대해서는 .*? 필요합니다[\.\n]){2,}",
        r"(이를 위해 법적 조언을 받으시기 바랍니다[\.\n]){2,}",
        r"((?:이 답변은|법률 전문가|법적 조언).*?){3,}",
        r"(이를 통해.*?신고가 정상적으로 이루어질 수 있습니다[\.\n]){2,}",
        r"(위자료 준비 과정에 대한 상담을 받으시기 바랍니다[\.\n]){2,}",
        r"(법적 고문이나 법원에서.*?받으시기 바랍니다[\.\n]){2,}"
    ]
    return any(re.search(pattern, text, flags=re.DOTALL) for pattern in patterns)

def clean_and_finalize(text):
    text = clean_text(text)
    text = deduplicate_text(text)
    text = remove_trailing_phrases(text)
    text = format_answer_blocks(text)
    return text


def is_case_summary_question(question: str) -> str | None:
    match = re.search(r"사건번호[:\s]*(\d+[가-힣]+\d+)", question)
    return match.group(1) if match else None

def build_case_summary_prompt(case_id: str, context: str) -> str:
    return f"""당신은 법률 문서 요약 AI입니다. 아래 사건번호 {case_id}에 대한 내용을 간결하고 명확하게 요약하세요.

1️⃣ 사건 개요
2️⃣ 핵심 쟁점
3️⃣ 판결 요지
4️⃣ 기타 참고 사항 (있다면)

📄 문서:
{context}

💬 요약:"""

# ✅ 일반 상담용 프롬프트 템플릿
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["user_info", "question", "context"],
    template="""
역할: 전문 가족법 변호사 (40년 경력, 이혼 및 가사소송 특화)

📌 당신은 다음 원칙을 기반으로 응답합니다:
- 민법 조항 또는 판례에 기반한 **법적 해석**을 제시합니다.
- 사용자의 가족 상황을 반영하여 **실질적인 조언**을 제공합니다.
- 답변은 아래 예시 형식을 철저히 따릅니다.

---

👤 의뢰인 정보:
- 혼인 상태: {user_info[marital_status]}
- 혼인 기간: {user_info[marriage_duration]}
- 이혼 진행 상태: {user_info[divorce_stage]}
- 자녀 정보: {user_info[children]}
- 가정폭력 경험 : {user_info[abuse_history]}
- 재산 범위: {user_info[property_range]}
- 민감 정보 포함 여부: {user_info[private_question]}

❓ 질문:
{question}

📄 참고 문서:
{context}

---

💬 [예시 형식] 다음과 같은 형식으로 답변을 작성하세요:

1. 법적 해석과 답변  
민법 제840조 제1호에 따르면, **배우자의 부정행위(외도)**는 재판상 이혼 사유에 해당합니다.

다만, 단순 외도 사실만으로 이혼이 인정되는 것은 아니며, 다음 요건을 함께 따집니다:
- 외도에 대한 **입증 가능한 증거**가 존재하는가? (사진, 문자, 위치기록 등)
- 외도 사실을 알고도 **용서하거나 재동거**한 정황이 있는가?
- 혼인을 유지하려는 **의사표현 또는 시도**가 있었는가?

---

2. 맞춤형 대응 전략  
- **증거 수집**: 외도 정황에 대한 문자, 위치 기록, SNS 메시지 등을 확보  
- **용서 유무 정리**: 동거 여부, 카톡 내역, 대화 기록 등을 정리해 소송 대응 준비  
- **전문가 상담**: 변호사와 함께 이혼 전략 및 위자료 청구 가능성 검토  
- **자녀/재산 고려**: 자녀가 있을 경우 양육 계획, 재산 분할 방안도 함께 준비

---

📋 참고 판례 요약:
- **대법원 2004므1234**: 반복된 외도는 명백한 혼인 파탄 사유로 이혼 인정  
- **대법원 2012므4567**: 외도 사실을 알면서 용서하고 동거한 경우, 이혼 청구가 기각될 수 있음

---

🟢 위와 같은 형식을 그대로 유지하여, 사용자 질문에 맞는 실제 답변을 작성하세요.

[출력 가이드라인]
- 최대한 간결하고 명확한 언어 사용
- 전문 용어는 일반인도 이해할 수 있게 설명
- 정보의 우선순위에 따라 구조화
- 시각적 구분을 위해 이모지, 들여쓰기, 줄바꿈 적극 활용
- 핵심 정보는 볼드체나 특별 강조로 표시
- 긴 텍스트는 간결하고 명확한 문장으로 압축하세요
- 반복 최소화


"""
)



# ✅ 질문 분류

def is_general_question_via_llm(question):
    try:
        res = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "다음 질문이 일반적인 법률 지식(법 조항 설명, 개념 정의, 절차 안내 등)에 대한 것인지 '예' 또는 '아니오'로만 답하세요."},
                {"role": "user", "content": question}
            ]
        )
        return "예" in res.choices[0].message.content.strip().lower()
    except Exception:
        return False

def build_prompt(user_info, question, context):
    return prompt_template.format(
        user_info=user_info,
        question=question,
        context=context
    )

def safe_utf8(text):
    return text.encode("utf-8", "surrogatepass").decode("utf-8", "ignore")
    

# ✅ Cosine 유사도 계산
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# ✅ 파인튜닝 초안 vs GPT 정제 유사도 비교
def is_similar_answer(draft: str, refined: str, threshold: float = 0.945) -> bool:
    embedding = OpenAIEmbeddings()
    vecs = embedding.embed_documents([draft, refined])
    return cosine_similarity(vecs[0], vecs[1]) > threshold

def filter_docs_with_gpt(question: str, docs: list) -> list:
    filtered = []
    for doc in docs:
        content = doc.page_content[:1000]  # 너무 길면 자르기
        meta = doc.metadata.get("사건번호", "문서")  # 문서 제목용

        check = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "당신은 가족법 전문가입니다. 다음 문서가 아래 질문과 법률적 관점에서 직접적인 관련이 있는지 판단하세요. "
                        "'예' 또는 '아니오'만 정확히 출력하세요."
                    )
                },
                {
                    "role": "user",
                    "content": f"질문: {question}\n\n문서 요약:\n{content}"
                }
            ]
        )

        result = check.choices[0].message.content.strip().lower()
        if "예" in result:
            filtered.append(doc)
    return filtered

def is_law_or_case_summary_question(message: str) -> bool:
    return bool(re.search(r"(민법\\s?제?\\d+조|형법\\s?제?\\d+조|조문|판례|사건번호)", message))


def filter_relevant_docs(docs, question, threshold=0.7):
    embedding = OpenAIEmbeddings()
    q_vec = embedding.embed_query(question)
    kept = []
    for doc in docs:
        text = (
            (doc.metadata.get("법률명") or "") + " " +
            (doc.metadata.get("조항번호") or "") + " " +
            (doc.metadata.get("사건번호") or "") + " " +
            doc.page_content
        )
        d_vec = embedding.embed_query(text)
        sim = cosine_similarity(q_vec, d_vec)
        if sim > threshold:
            kept.append(doc)
    return kept

def extract_single_answer(output_text):
    # "### 답변:" 이후만 추출 (있다면)
    if "### 답변:" in output_text:
        return output_text.split("### 답변:")[-1].strip()
    return output_text.strip()

# ✅ 메인 챗봇 함수
def chat_fn(message):
    # ✅ 1. 법 조문 또는 판례 요약 질문 처리
    if is_law_or_case_summary_question(message):
        gpt_response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": (
                    "당신은 법률 요약 전문가입니다. 다음과 같은 4단계 형식으로 출력하세요:\n\n"
                    "1️⃣ 관련 법률 조항\n2️⃣ 주요 판례 요약\n3️⃣ 일반적인 법적 해석\n4️⃣ 맞춤형 대응 전략\n\n"
                    "- 민법 또는 가족법 조항을 명확하게 제시\n"
                    "- 관련된 대법원 판례나 주요 사례를 요약\n"
                    "- 상황에 따른 법적 해석을 설명\n"
                    "- 사용자가 고려해야 할 대응 전략을 명확하게 정리\n"
                    "- 중복, 반복 문구는 피하고 논리적으로 간결하게 서술"
                )},
                {"role": "user", "content": message}
            ]
        )
        answer = gpt_response.choices[0].message.content.strip()
        chat_history.append((message, f"{answer}\n\n🟠 (파인튜닝 모델 + GPT)"))
        return chat_history, ""

    # ✅ 2. 사건번호 기반 요약 처리
    case_id = is_case_summary_question(message)
    docs = retriever.get_relevant_documents(message)
    docs = filter_relevant_docs(docs, message)

    if case_id:
        filtered_docs = [doc for doc in docs if doc.metadata.get("사건번호") == case_id]
        if filtered_docs:
            docs = filtered_docs
        context = format_documents_with_metadata(docs)
        prompt = build_case_summary_prompt(case_id, context)

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "당신은 법률 문서 요약 전문가입니다."},
                {"role": "user", "content": prompt}
            ]
        )
        answer = extract_single_answer(response.choices[0].message.content)
        chat_history.append((message, f"{answer}\n\n🟠 (파인튜닝 모델 + GPT)\n\n📋 참고 문서:\n{context}"))
        return chat_history, ""

    # ✅ 3. 일반 질문 처리 + 세션 기반 유사 질문 검색
    related_context = ""
    if chat_history:
        q_vec = embedding.embed_query(message)
        for past_q, past_a in reversed(chat_history[-5:]):
            past_vec = embedding.embed_query(past_q)
            sim = cosine_similarity(q_vec, past_vec)
            if sim > 0.85:
                related_context = ""
                break

    context = related_context + "\n\n" + format_documents_with_metadata(docs) if docs else related_context
    prompt = build_prompt(user_info, message, context)

    output = pipe(prompt, max_new_tokens=1500, temperature=0.7, do_sample=True)[0]["generated_text"]
    draft = extract_single_answer(output[len(prompt):]).strip()

    gpt_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": (
                "당신은 20년 경력의 가족법 전문 변호사입니다. 아래는 파인튜닝 모델이 생성한 응답 초안입니다.\n"
                "아래 질문에 대해 다음 4단계 형식으로 출력하세요:\n\n"
                "1️⃣ 관련 법률 조항\n2️⃣ 주요 판례 요약\n3️⃣ 일반적인 법적 해석\n4️⃣ 맞춤형 대응 전략\n\n"
                "- 민법 또는 가족법 조항을 명확하게 제시\n"
                "- 관련된 대법원 판례나 주요 사례를 요약\n"
                "- 상황에 따른 법적 해석을 설명\n"
                "- 사용자가 고려해야 할 대응 전략을 명확하게 정리\n"
                "- 중복, 반복 문구는 피하고 논리적으로 간결하게 서술"
            )},
            {"role": "user", "content": f"질문: {message}\n\n파인튜닝 모델 초안 응답:\n{draft}"}
        ]
    )
    refined = gpt_response.choices[0].message.content.strip()
    mark = "🟢 (파인튜닝 모델)" if is_similar_answer(draft, refined) else "🟠 (파인튜닝 모델 + GPT)"
    final_answer = f"{refined}\n\n{mark}"
    doc_section = f"\n\n📋 참고 문서:\n{context}" if context else ""

    chat_history.append((message, final_answer + doc_section))
    return chat_history, ""



# ✅ Gradio UI
def build_ui():
    with gr.Blocks() as demo:
        gr.Markdown("## 📚 LawQuick")

        with gr.Row():
            with gr.Column():
                gr.Markdown("**👤 기본 정보**")
                marital_status = gr.Dropdown(label="1. 혼인 상태", choices=["기혼", "이혼", "별거", "사실혼"])
                m_priv = gr.Checkbox(label="비공개 처리")
                marriage_duration = gr.Textbox(label="2. 혼인 기간 (예: 10년)")
                d_priv = gr.Checkbox(label="비공개 처리")
                divorce_stage = gr.Dropdown(label="3. 이혼 진행 상태", choices=["이혼 고려 중", "이혼 준비 중", "이혼 진행 중", "이미 이혼함"])
                ds_priv = gr.Checkbox(label="비공개 처리")

            with gr.Column():
                gr.Markdown("**👨‍👩‍👧‍👦 가족 및 재산 정보**")
                children = gr.Textbox(label="4. 자녀 정보 (예: 2명, 5세/8세)")
                c_priv = gr.Checkbox(label="비공개 처리")
                abuse_history = gr.Radio(label="5. 가정폭력 또는 정신적 고통 경험", choices=["없음", "있음"], value="없음")
                a_priv = gr.Checkbox(label="비공개 처리")
                property_range = gr.Dropdown(label="6. 재산 범위", choices=["1천만 원 미만", "1천만~5천만 원", "5천만~1억 원", "1억 원 이상"])
                p_priv = gr.Checkbox(label="비공개 처리")
                private_question = gr.Checkbox(label="🛑 질문에 민감한 개인정보가 포함되어 있음", value=False)

        confirm_btn = gr.Button("✅ 사용자 정보 등록")
        user_info_status = gr.Markdown()
        chatbot = gr.Chatbot()
        msg = gr.Textbox(label="💬 질문 입력", placeholder="예: 남편이 폭행하면 위자료 얼마나 받을 수 있나요?", lines=2)
        ask_btn = gr.Button("질문하기")
        clear_btn = gr.Button("초기화")

        confirm_btn.click(register_user,
                          inputs=[marital_status, m_priv, marriage_duration, d_priv, divorce_stage, ds_priv,
                                  children, c_priv, abuse_history, a_priv, property_range, p_priv, private_question],
                          outputs=[user_info_status])
        ask_btn.click(chat_fn, inputs=[msg], outputs=[chatbot, msg])
        clear_btn.click(lambda: ([], ""), None, [chatbot, msg])

    return demo

if __name__ == "__main__":
    demo = build_ui()
    demo.launch(share=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Device set to use cuda:0
/tmp/ipykernel_2851/1957843816.py:468: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be

* Running on local URL:  http://127.0.0.1:7873
* Running on public URL: https://ff16a9fa3413829833.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
